In [3]:
from mlrun import new_function, NewTask, get_run_db, mlconf, mount_v3io, new_model_server, builder
import kfp
from kfp import dsl
import os
from os.path import isfile, join

In [4]:
mlconf.dbpath = 'http://mlrun-api:8080'

In [25]:
# Environment vars to be set by Nuclio
SCRIPTS_LOCATION = os.getenv('SCRIPTS_LOCATION','/User/igzshare/nuclio_trigger_kfp')

In [26]:
#Create MLRun function to run the spark-job on the kubernetes cluster
k8sjob_fn = new_function(kind='job', command=os.path.join(SCRIPTS_LOCATION,'training.py')).apply(mount_v3io(name='v3io', remote='~/', mount_path='/User', access_key=os.getenv('V3IO_ACCESS_KEY'),
      user=os.getenv('V3IO_USERNAME')))

In [27]:
k8sjob_fn.spec.image="mlrun/mlrun"

In [39]:
from os import path
out = mlconf.artifact_path or os.path.join(SCRIPTS_LOCATION,'data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

In [49]:
print(path.abspath('./data'))
#k8sjob_fn.with_limits(mem=SPARK_SPEC_MEM)
#k8sjob_fn.with_requests(cpu=SPARK_SPEC_CPU)
#k8sjob_fn.spec.replicas = SPARK_SPEC_REPLICAS

/User/igzshare/nuclio_trigger_kfp/data


In [41]:
@dsl.pipeline(
    name='Kubeflow pipeline triggered by Nuclio',
    description='Trigger run'
)
def example_pipeline(
   p1 = [1,2,3,4,5,6],
   p2 = [9,8,6,5,4,3]
):
    # Use the same fn definition, but run different functions.
    # fn2 is a definition from the external notebook
    f1 = k8sjob_fn.as_step(NewTask(artifact_path=artifact_path), name='training').apply(mount_v3io(name='v3io', remote='~/', mount_path='/User', access_key=os.getenv('V3IO_ACCESS_KEY'),
      user=os.getenv('V3IO_USERNAME')))
    

In [42]:
client = kfp.Client(namespace='default-tenant')
p1 = [1,2,3,4,5,6]
p2 = [9,8,6,5,4,3]
#arguments = {'p1': p1 ,'p2': p2}
arguments={}

# Record pipeline deployment in KV

In [43]:
import uuid
import v3io.dataplane

In [44]:
v3io_client = v3io.dataplane.Client(max_connections=1)

In [45]:
def record_pipeline_id(run_id):
    record_id=str(run_id)
    v3io_client.put_item(container=os.getenv('MONITOR_CONTAINER','bigdata'),
                         path=os.path.join(os.getenv('MONITOR_TABLE','kubeflow_runs'),record_id),
                         attributes={
                             'id': record_id,
                             'status' : 'started',}
                        )

# get the Kubeflow run_id
Note: This notebook was written to invoke the deployment as an mlrun function.

In [46]:
def handler(context,event):
    run_id=str(uuid.uuid4())
    run_result = client.create_run_from_pipeline_func(example_pipeline, arguments, run_name='KpfNuclioTriggered-'+run_id, experiment_name='nucliotrigger')
    record_pipeline_id(run_result.run_id)
    return run_result.run_id

In [50]:
# nuclio: ignore
run_id=str(uuid.uuid4())
run_result = client.create_run_from_pipeline_func(example_pipeline, arguments, run_name='KpfNuclioTriggered-'+run_id, experiment_name='nucliotrigger')
record_pipeline_id(run_result.run_id)